In [ ]:
# Transfer Learning

In [ ]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device set to {device}")
!nvidia-smi

### import previously written code

In [ ]:
import matplotlib as plt
import torch
import torchvision

try:
    from torchinfo import summary
except:
    print("[INFO] Could not find torchinfo...installing now..")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular.going_modular import data_setup, engine
except:
    print("[INFO] Could not find going_modular scripts...downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

### downloading pizz_steak_sushi images

In [ ]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)
    
# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

# manual data transforms

In [ ]:
# torchvision.models contains pretrains models
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.255])

manual_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [ ]:
from going_modular.going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=32,
    num_workers=2
)
train_dataloader, test_dataloader, class_names

# auto create transforms for pretrained



In [ ]:
from torchvision.models import EfficientNet_B0_Weights

weights = EfficientNet_B0_Weights.DEFAULT
weights

In [ ]:
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
from going_modular.going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=auto_transforms,
    batch_size=32,
    num_workers=2
)
train_dataloader, test_dataloader, class_names

### getting a pretrained model

1. PyTorch domain libraries
2. Libraries like `timm` (torch image models)
3. HuggingFace Hub 
4. Paperswithcode

### When choosing a model:
1. speed - how fast does it run
2. size - how large is the model
3. performance - how well does it fit your the problem your solving?

Where will the model live?
* on a device?
* on a server?

## Setting up pretrained model

In [ ]:
# old method
# model = torchvision.models.efficientnet_b0(pretrain=True)

# new method
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model

# Summary of Model with torchinfo

In [ ]:
from torchinfo import summary

summary(model=model,
        input_size=(1,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]) # batchsize, color channels, height, width

### Freezing Base model

In [ ]:
# freezing base layers
for param in model.features.parameters():
    # print(param)
    param.requires_grad = False

In [ ]:
# update classifier head of model
from torch import nn
torch.manual_seed(42)
torch.cuda.manual_seed(42)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names), 
              bias=True).to(device)
)
model.classifier

In [ ]:
from torchinfo import summary

summary(model=model,
        input_size=(1,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]) # batchsize, color channels, height, width

In [ ]:
# train model
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from going_modular.going_modular import engine

torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer
start_time = timer()

results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=5,
                       device=device)

end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
# plot loss curves
try:
    from helper_functions import plot_loss_curves
    print("failed to download helper functions")
except:
    print(f"[INFO] could not find helper functions, downloading...")
    with open("helper_functions.py", "wb") as f:
      import requests
      request = request.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/helper_functions.py')
      f.write(request.content)
    from helper_functions import plot_loss_curves

plot_loss_curves(results)
